In [2]:
import pickle

with open('soccerPerformance3.pickle', 'rb') as handle:
    b3 = pickle.load(handle)
    
cnt = 0
teamDic = dict()
teams = b3.teams




def createDataSet(gameInfo, season):
    x = []
    y = []
    cnt = 0
    #this code block is meant to fix a mistake in the definition of my data structure, should really make performances
    # a dictionary which each team name as a key, and the performances as a val. 
    for team in teams:
        teamDic[team] = cnt
        cnt += 1
    for game in gameInfo:
        example = []
        stage = game[0]
        res = game[5]
        #the result of the game will constitute our y set. 0 = home loss, 1 = home draw, 3 = home win.
        y.append(res)
        home = game[1]
        away = game[2]
        homeLoc = teamDic[home]
        awayLoc = teamDic[away]
        #a training example will be a 10 * 826, with the home teams most recent 5 games as the first 5 values
        #followed by the away teams most recent 5 games.
        for i in range((stage-6), (stage - 1)):
            game = season.performances[homeLoc].performace[i]
            example.append(game)
        for j in range((stage-6), (stage - 1)):
            game = season.performances[awayLoc].performace[i]
            example.append(game)
        x.append(example)
    return x, y

In [4]:
#creates a flat version of the x as a 1 x 8240 matrix
def createDataSet(gameInfo, season):
    x = []
    y = []
    cnt = 0
    #this code block is meant to fix a mistake in the definition of my data structure, should really make performances
    # a dictionary which each team name as a key, and the performances as a val. 
    for team in teams:
        teamDic[team] = cnt
        cnt += 1
    for game in gameInfo:
        example = []
        stage = game[0]
        res = game[5]
        #the result of the game will constitute our y set. 0 = home loss, 1 = home draw, 3 = home win.
        y.append(res)
        home = game[1]
        away = game[2]
        homeLoc = teamDic[home]
        awayLoc = teamDic[away]
        #a training example will be a 10 * 826, with the home teams most recent 5 games as the first 5 values
        #followed by the away teams most recent 5 games.
        for i in range((stage-6), (stage - 1)):
            game = season.performances[homeLoc].performance[i]
            #first two items in game are the stage and away team api Id which I don't use in this
            # model
            game1 = game[2:]
            for d in game1:
                example.append(d)
        for j in range((stage-6), (stage - 1)):
            game = season.performances[awayLoc].performance[i]
            game1 = game[2:]
            for d in game1:
                example.append(d)
        x.append(example)
    
    return x, y

#changes the output of a game to a one hot encoding [win, tie, loss]
def resOneHot(y):
    if y == 3:
        return [1 , 0, 0]
    elif y == 1:
        return [0, 1, 0]
    else:
        return [0, 0, 1]

#creates a flat version of the x as a 1 x 8240 matrix and the y's as one hot vectors.
def createDataSet2(gameInfo, season):
    x = []
    y = []
    cnt = 0
    #this code block is meant to fix a mistake in the definition of my data structure, should really make performances
    # a dictionary which each team name as a key, and the performances as a val. 
    for team in teams:
        teamDic[team] = cnt
        cnt += 1
    for game in gameInfo:
        example = []
        stage = game[0]
        res = game[5]
        #the result of the game will constitute our y set. 0 = home loss, 1 = home draw, 3 = home win.
        y.append(resOneHot(res))
        home = game[1]
        away = game[2]
        homeLoc = teamDic[home]
        awayLoc = teamDic[away]
        #a training example will be a 10 * 826, with the home teams most recent 5 games as the first 5 values
        #followed by the away teams most recent 5 games.
        for i in range((stage-6), (stage - 1)):
            game = season.performances[homeLoc].performance[i]
            #first two items in game are the stage and away team api Id which I don't use in this
            # model
            game1 = game[2:]
            for d in game1:
                example.append(d)
        for j in range((stage-6), (stage - 1)):
            game = season.performances[awayLoc].performance[i]
            game1 = game[2:]
            for d in game1:
                example.append(d)
        x.append(example)
    
    return x, y

In [5]:
trainGames = b3.info[50:330]
testGames = b3.info[330:]

xTrain, yTrain = createDataSet(trainGames, b3)
xTest, yTest = createDataSet(testGames, b3)

In [6]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
clf = RandomForestClassifier(n_estimators=100)
clf.fit(xTrain, yTrain)
print(clf.score(xTrain, yTrain))
print(clf.score(xTest, yTest))
yVals = clf.predict(xTest)
print(yVals)
print(yTest)

1.0
0.44
[3 3 3 3 3 3 3 3 0 3 3 0 3 3 3 3 3 0 3 3 3 0 3 3 1 3 3 3 3 3 3 3 0 3 3 3 3
 3 3 3 3 3 3 3 0 3 3 1 3 3]
[1, 0, 0, 1, 1, 3, 3, 0, 0, 0, 0, 0, 1, 3, 1, 3, 3, 1, 1, 3, 3, 1, 3, 1, 3, 3, 1, 3, 3, 0, 1, 1, 3, 3, 3, 1, 0, 3, 0, 0, 3, 1, 3, 3, 3, 3, 3, 1, 1, 1]


In [7]:
import tensorflow as tf
xTrain2, yTrain2 = createDataSet2(trainGames, b3)
xTest2, yTest2 = createDataSet2(testGames, b3)

x = tf.placeholder(tf.float32, shape=[None, 8240])
W = tf.Variable(tf.zeros([8240,3]))
b = tf.Variable(tf.zeros([3]))
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

NameError: name 'y_conv' is not defined

In [11]:
#this is still in progress working on building neural network
import random as rand
def genBatch(x, y, n =25):
    x1 = []
    y1 = []
    a = range(len(y))
    rand.shuffle(a)
    for i in range(n):
        j = a[i]
        x1.append(x[j])
        y1.append(y[j])
    return x1, y1
    
import tensorflow as tf
sess = tf.InteractiveSession()
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 8, 1, 1],
                        strides=[1, 8, 1, 1], padding='SAME')

x = tf.placeholder(tf.float32, shape=[None, 8240])
y_ = tf.placeholder(tf.float32, shape=[None, 3])

W_conv1 = weight_variable([32, 1, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,824,10,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([16, 1, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([13 * 10 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1,13 * 10 * 64 ])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 3])
b_fc2 = bias_variable([3])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.initialize_all_variables())
for i in range(200):
    x_batch, y_batch = genBatch(xTrain2, yTrain2) 
  
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:x_batch, y_: y_batch, keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: xTrain2, y_: yTrain2, keep_prob: 1.0}))

Instructions for updating:
Use `tf.global_variables_initializer` instead.


TypeError: 'range' object does not support item assignment

In [10]:
a = [1, 2, 3]
a.shuffle()
print(a)

AttributeError: 'list' object has no attribute 'shuffle'

SyntaxError: invalid syntax (<ipython-input-14-a2b246f98dc7>, line 1)